In [20]:
#basic RNN
import tensorflow as tf

n_inputs =3
n_neurons = 5

X0 =  tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) +b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()

In [21]:
import numpy as np

X0_batch = np.array([[0,1,2], [3,4,5], [6,7,8], [9,0,1]])
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0:X0_batch, X1:X1_batch})
    

In [22]:
print(Y0_val)

[[ 0.736362    0.56424904 -0.9928428   0.99697137  0.9957681 ]
 [ 0.9999985   0.98514026 -1.          1.          1.        ]
 [ 1.          0.9995978  -1.          1.          1.        ]
 [ 0.99999887  0.9990503  -0.99999994  0.99999994  0.99580586]]


In [23]:
print(Y1_val)

[[ 1.          0.9999874  -1.          1.          1.        ]
 [ 0.69753206  0.6118082  -0.90616906  0.9795825  -0.8114883 ]
 [ 1.          0.9986853  -1.          1.          1.        ]
 [ 0.9999879   0.9524181  -0.99999994  0.99996644  0.9973081 ]]


## BasicRNNCell

In [24]:
tf.reset_default_graph()
# static unrolling through time

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])
# we create input palceholders

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
#  we create BasicRNNCell which you can think of as a factory that creates copies of the cell to build the unrolled RNN

output_seqs,states = tf.contrib.rnn.static_rnn(
                     basic_cell, [X0, X1], dtype=tf.float32)
# we call static rnn which calls the BasicRNNCell creating two copies of cell


Y0, Y1 = output_seqs

In [25]:
# if there wer 50 time steps, it wouldnot be ery convenient to have ot 
# define 50 input placeholders and 50 output tensors

print(Y0)
print(Y1)



Tensor("rnn/basic_rnn_cell/Tanh:0", shape=(?, 5), dtype=float32)
Tensor("rnn/basic_rnn_cell/Tanh_1:0", shape=(?, 5), dtype=float32)


In [26]:
# using the packing sequences
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0:X0_batch, X1:X1_batch})

In [27]:
Y0_val

array([[-0.6220045 ,  0.3434742 ,  0.5967921 ,  0.5467581 , -0.11334981],
       [-0.8740386 ,  0.9972768 ,  0.987609  ,  0.9661833 , -0.81035984],
       [-0.96196574,  0.9999924 ,  0.9996922 ,  0.99798304, -0.972817  ],
       [-0.27229077,  0.9997892 ,  0.99158883,  0.99976593, -0.99935156]],
      dtype=float32)

In [28]:
Y1_val

array([[-0.62965757,  0.99999994,  0.9999669 ,  0.9991317 , -0.9964192 ],
       [ 0.6258266 ,  0.7319028 ,  0.891579  , -0.2367793 , -0.00984494],
       [ 0.22992855,  0.99999607,  0.9997939 ,  0.9622914 , -0.95374256],
       [ 0.55169165,  0.9969307 ,  0.97981477,  0.08551132, -0.7642403 ]],
      dtype=float32)

In [30]:
# from tensorflow_graph_in_jupyter import show_graph

# show_graph(tf.get_default_graph())

## Packing Sequences

In [31]:
# the following cide builds the same RNN again but thistime it takes asingle input 
# placeholder of hsape [None, n_steps, n_inputs ] where the first  fimensions is the mini batch size\
# Then it extract thelist of input sequences for each time step
# X_seqs is a Python list of n_steps tensors of shape [None,n_inputs]
# where once again the first dimension is the mini batch size
# To do this, we first swap using transpose function so that the time steps are now the first dimension 
# then we use the unstack function. The next twolines are the sam
# finally we merge the the output tensors into a single tensor using stack() function
# and we sqap the first two fimension to get a final outputs tensor
# of shape [None, n_steps, n_neurons]

n_steps =2
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps,n_inputs])
X_seqs = tf.unstack(tf.transpose(X,perm=[1,0,2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1,0,2])

In [32]:
init = tf.global_variables_initializer()

In [34]:
X_batch= np.array([
    [[0,1,2], [9,8,7]],
    [[3,4,5], [0,0,0]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]]
])
g = tf.Graph()
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X:X_batch})

# this stillgets one graph containing unit cell per time step
# If there were 50 time steps the graph would look pretty ugly
print(outputs_val)

# with large graph we will also get out of memory errors so that it can
# use them to compute gradients during the reverse pass
# thats why we using dynamic rnn

[[[ 0.80800104 -0.25818652  0.88132834  0.79292494 -0.08273872]
  [-0.5822327  -0.9999026   0.999995   -0.657447   -0.96924794]]

 [[ 0.8161503  -0.9309901   0.99883807  0.84217244 -0.7897056 ]
  [ 0.5496408  -0.85100687  0.21461645 -0.20366207  0.8525351 ]]

 [[ 0.82398736 -0.9956768   0.9999893   0.88048846 -0.9679276 ]
  [-0.33215857 -0.999035    0.9979041  -0.7774459  -0.4956271 ]]

 [[-0.9999593  -0.999779    0.14563009 -0.9997508  -0.93437034]
  [-0.87933636 -0.6990654  -0.65328014 -0.6297835  -0.6142405 ]]]


In [35]:
print(np.transpose(outputs_val, axes=[1,0,2])[1])

[[-0.5822327  -0.9999026   0.999995   -0.657447   -0.96924794]
 [ 0.5496408  -0.85100687  0.21461645 -0.20366207  0.8525351 ]
 [-0.33215857 -0.999035    0.9979041  -0.7774459  -0.4956271 ]
 [-0.87933636 -0.6990654  -0.65328014 -0.6297835  -0.6142405 ]]


In [44]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

In [47]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as ses:
    init.run()
    outputs_val = outputs.eval(feed_dict=)